# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
categorical_columns = [column for column in df.columns if df[column].dtype=='O' and column!='loan_status']
categorical_columns

['homeowner']

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['homeowner'] =  le.fit_transform(df['homeowner'])

In [6]:
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,1,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,1,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,2,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,1,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,0,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [7]:
# Create our features
X =  df.drop('loan_status' , axis = 1)

# Create our target
y = df['loan_status']

In [8]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = 1, stratify =  y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaller = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
x_scaller = scaller.fit(x_train)

In [13]:
x_train.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
25729,10100.0,7.402,0,50300,0.403579,4,1,20300
20176,9200.0,7.049,0,46900,0.360341,3,0,16900
55080,8800.0,6.870,0,45200,0.336283,3,0,15200
10410,8900.0,6.919,0,45700,0.343545,3,0,15700
63110,8500.0,6.731,1,43900,0.316629,3,0,13900


In [14]:
# Scale the training and testing data
x_trainscaller = x_scaller.transform(x_train)
x_testscaller = scaller.transform(x_test)

In [15]:
x_trainscaller

array([[ 0.14045627,  0.1230916 , -0.90997878, ...,  0.09115188,
         1.04495557,  0.12859477],
       [-0.28969527, -0.27393966, -0.90997878, ..., -0.43434275,
        -0.67471435, -0.27772578],
       [-0.48087373, -0.47526712, -0.90997878, ..., -0.43434275,
        -0.67471435, -0.48088605],
       ...,
       [-1.48456066, -1.46728289, -0.90997878, ..., -1.48533201,
        -0.67471435, -1.47278621],
       [-0.28969527, -0.26831599, -0.90997878, ..., -0.43434275,
        -0.67471435, -0.26577518],
       [-1.05440912, -1.05787955, -0.90997878, ..., -0.95983738,
        -0.67471435, -1.05451506]])

# Simple Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(x_train, y_train)

LogisticRegression(random_state=1)

In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(x_testscaller)
balanced_accuracy_score(y_test, y_pred)

0.8041461911615757

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,7258,11501


In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      1.00      0.61      0.15      0.78      0.63       625
   low_risk       1.00      0.61      1.00      0.76      0.78      0.59     18759

avg / total       0.97      0.63      0.98      0.74      0.78      0.59     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [20]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(random_state = 1)
x_oversample, y_oversample = oversample.fit_resample(x_trainscaller, y_train)

# View the count of target classes with Counter
Counter(y_oversample)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [21]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(x_oversample, y_oversample)

LogisticRegression(random_state=1)

In [22]:
# Calculated the balanced accuracy score
y_pred_oversample = model.predict(x_testscaller)
balanced_accuracy_score(y_test, y_pred_oversample)

0.9946414201183431

In [23]:
# Display the confusion matrix
cm_oversample = confusion_matrix(y_test, y_pred_oversample)
cm_oversample = pd.DataFrame(
    cm_oversample, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_oversample

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_oversample))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [25]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 1)
x_smote, y_smote = smote.fit_resample(x_trainscaller, y_train)

# View the count of target classes with Counter
Counter(y_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [26]:
# Train the Logistic Regression model using the resampled data
modelsmote = LogisticRegression(solver='lbfgs', random_state=1)
modelsmote.fit(x_smote, y_smote)

LogisticRegression(random_state=1)

In [27]:
# Calculated the balanced accuracy score
y_smote = model.predict(x_testscaller)
balanced_accuracy_score(y_test, y_smote)

0.9946414201183431

In [28]:
# Display the confusion matrix
cm_smotesample = confusion_matrix(y_test, y_smote)
cm_smotesample = pd.DataFrame(
    cm_smotesample, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_smotesample

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cluster = ClusterCentroids(random_state =1)
cluster_x, cluster_y = cluster.fit_resample(x_trainscaller, y_train)
# View the count of target classes with Counter
Counter(cluster_y)

In [31]:
Counter(cluster_y)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [32]:
# Train the Logistic Regression model using the resampled data
model_cluster = LogisticRegression(solver='lbfgs', random_state=1)
model_cluster.fit(cluster_x, cluster_y)

LogisticRegression(random_state=1)

In [33]:
# Calculate the balanced accuracy score
y_cluster = model_cluster.predict(x_testscaller)
balanced_accuracy_score(y_test, y_cluster)

0.9932813049736127

In [34]:
# Display the confusion matrix
cm_cluster = confusion_matrix(y_test, y_cluster)
cm_cluster = pd.DataFrame(
    cm_cluster, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_cluster

,Predicted High Risk,Predicted Low Risk
Actual High Risk,620,5
Actual Low Risk,102,18657


In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_cluster))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [38]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN 
smoteenn = SMOTEENN(random_state = 1)
smooth_x, smooth_y = smoteenn.fit_resample(x_trainscaller, y_train)

# View the count of target classes with Counter
Counter(smooth_y)

Counter({'high_risk': 55667, 'low_risk': 55917})

In [39]:
# Train the Logistic Regression model using the resampled data
smoteenn = LogisticRegression(solver='lbfgs', random_state=1)
smoteenn.fit(smooth_x, smooth_y)

LogisticRegression(random_state=1)

In [41]:
# Calculate the balanced accuracy score
smoothpredict = smoteenn.predict(x_testscaller)
balanced_accuracy_score(y_test, smoothpredict)

0.9946414201183431

In [42]:
# Display the confusion matrix
cm_smooth = confusion_matrix(y_test, smoothpredict)
cm_smooth = pd.DataFrame(
    cm_smooth, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_smooth

,Predicted High Risk,Predicted Low Risk
Actual High Risk,622,3
Actual Low Risk,111,18648


In [43]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, smoothpredict))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   The combined model has a slight heigher accuracy score.

2. Which model had the best recall score?

    All models score the same.

3. Which model had the best geometric mean score?

    All models are pretty close.
